In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [91]:
# Importing the files
y_train = pd.read_csv('./data/dengue_labels_train.csv')
X_train = pd.read_csv('./data/dengue_features_train.csv')
X_test = pd.read_csv('./data/dengue_features_test.csv')

In [92]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression # as a baseline
from sklearn.preprocessing import StandardScaler

In [93]:
# data cleaning training set
df = pd.concat([y_train['total_cases'], X_train], axis=1)
df = df.dropna()
y_train = df['total_cases']
X_train = df.drop(columns=['total_cases'])

# data cleaning trainset
X_train['Month'] = pd.DatetimeIndex(X_train['week_start_date']).month
X_train = X_train.drop(columns=['year', 'weekofyear', 'week_start_date', 'city'])
for i in X_train.columns:
    X_train[i].fillna(X_train.groupby('Month')[i].transform('median'), inplace=True)

# data cleaning testset
X_test['Month'] = pd.DatetimeIndex(X_test['week_start_date']).month
X_test = X_test.drop(columns=['year', 'weekofyear', 'week_start_date', 'city'])
for i in X_test.columns:
    X_test[i].fillna(X_test.groupby('Month')[i].transform('median'), inplace=True)

#Baseline LinearRegression
pipe_lr = Pipeline([('scaler', StandardScaler()), ('lr', LinearRegression())])

pipe_lr.fit(X_train, y_train)
lr_predict = pipe_lr.predict(X_test)

In [94]:
subm = pd.read_csv('./submission_format.csv')
labels = pd.DataFrame({'total_cases': pd.Series(lr_predict)})
subm.loc[:, 'total_cases'] = labels.astype(int)
subm.to_csv('test.csv', index=False)